In [11]:
# Initial imports.
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.datasets import make_classification

In [34]:
# Loading data
# file_path = 
# df_loans = pd.read_csv(file_path)
# df_loans.head()
### ---------------------PLACEHOLDER----------------------
### The final csv will be uploaded to github, therefore the raw csv will be accessed using the github url
### This is a test dataframe
X, y = make_classification(random_state=18, n_classes=2, n_samples=400, n_features=300)

array([[ 1.44144246,  1.08438653, -0.28190824, ..., -2.47909796,
        -0.58588219, -0.43669044],
       [ 0.55355925, -0.16443415,  0.32335895, ..., -0.27532982,
         0.01003574, -0.3267483 ],
       [ 1.94081009, -0.88442602,  0.14781557, ..., -0.04514866,
         0.58175401, -1.44246295],
       ...,
       [-1.70460861,  0.70245195, -0.44440221, ...,  0.03976029,
         0.89482372, -1.15734208],
       [-1.03801954,  0.67329629,  0.85502538, ...,  0.2359358 ,
         0.58547744, -0.73437052],
       [-0.8859545 ,  1.58674044, -1.82532741, ..., -0.96601934,
         0.10654035,  0.35602187]])